In [19]:
import highspy
import numpy as np
import pandas as pd
from math import radians, cos, sin, asin, sqrt

h = highspy.Highs()

In [20]:
# Extract the postal codes that has available data
post_codes = pd.read_csv('Postcodes.csv')
post_code_list = []

for i in range(post_codes.shape[0]):
    if post_codes.iloc[i,2] == 1.0:
        post_code_list.append(post_codes.iloc[i,1])


# Extract the council area that has available data
council = pd.read_csv('CouncilAreas.csv')
council_list = []

for i in range(council.shape[0]):
    if council.iloc[i,2] == 1.0:
        council_list.append(council.iloc[i,1])

In [16]:
# Load the cardiac arrest data as well as AED location
cardiac_arrest = pd.read_csv('Cardiac arrests.csv')
AED = pd.read_csv("AEDs.csv")

def haversine(long1, lat1, long2, lat2):
    long1 = radians(long1)
    lat1 = radians(lat1)
    long2 = radians(long2)
    lat2 = radians(lat2)
    
    # Haversine formula
    dlong = long2 - long1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlong/2)**2
    c = 2*asin(sqrt(a))
    r = 6371 # Radius of earth in kilometers
    return c*r*1000

cardAED_prod = np.zeros((cardiac_arrest.shape[0] * AED.shape[0], 7))

for i in range(cardiac_arrest.shape[0]):
    for j in range(AED.shape[0]):
        cardAED_prod[i*AED.shape[0]+j][0] = i
        cardAED_prod[i*AED.shape[0]+j][1] = j
        cardAED_prod[i*AED.shape[0]+j][2] = cardiac_arrest.iloc[i, 2]
        cardAED_prod[i*AED.shape[0]+j][3] = cardiac_arrest.iloc[i, 1]
        cardAED_prod[i*AED.shape[0]+j][4] = AED.iloc[j, 3]
        cardAED_prod[i*AED.shape[0]+j][5] = AED.iloc[j, 2]
        cardAED_prod[i*AED.shape[0]+j][6] = haversine(cardiac_arrest.iloc[i, 1], cardiac_arrest.iloc[i, 2], AED.iloc[j, 2], AED.iloc[j, 3])

valid_card = [] # Contain all the cardiac arrest locations that are not covered by any AED
coverage_distance = 100

card_counter = 0
AED_counter = 0
while card_counter < cardiac_arrest.shape[0]:
    if AED_counter == AED.shape[0]:
        row = [card_counter, cardiac_arrest.iloc[card_counter, 2], cardiac_arrest.iloc[card_counter, 1]]
        valid_card.append(row)
        card_counter += 1
        AED_counter = 0
        continue
    if cardAED_prod[card_counter*AED.shape[0] + AED_counter][6] < coverage_distance:
        card_counter += 1
        AED_counter = 0
    else:
        AED_counter += 1

In [17]:
v_cardiac = pd.DataFrame(valid_card, columns=['id', 'lattitude', 'longitude'])
potential = pd.read_csv("PotentialLocations.csv")

postcode_of_interest = "DD10"
council_of_interest = ""
potential = potential[potential['Postcode'].str.contains(postcode_of_interest, regex=True)]

del potential['fid']
del potential['x']
del potential['y']
del potential['Postcode']
del potential['Council']


In [12]:
v_cardiac = pd.DataFrame(valid_card, columns=['id', 'lattitude', 'longitude'])
potential = pd.read_csv("Potential locations.csv")

distMatrix = np.zeros((v_cardiac.shape[0], potential.shape[0]))
for i in range(v_cardiac.shape[0]):
    for j in range(potential.shape[0]):
        distMatrix[i][j] = haversine(v_cardiac.iloc[i, 2], v_cardiac.iloc[i, 1], potential.iloc[j, 2], potential.iloc[j, 1])

deleted_AED = []
for j in range(potential.shape[0]):
    for i in range(v_cardiac.shape[0]):
        if distMatrix[i][j] <= coverage_distance:
            break
        if i == v_cardiac.shape[0] - 1:
            deleted_AED.append(j)

deleted_AED.sort(reverse=True)
potential = potential.drop(deleted_AED, axis=0, errors='ignore')
for AED in deleted_AED:
    distMatrix = np.delete(distMatrix, AED, axis=1)

In [13]:
dM = pd.DataFrame(distMatrix)
N = [5, 10, 15, 20, 50, 100]
covered = (dM <= coverage_distance).astype(int)
n_cases = covered.shape[0] # The same as cardiac_arrest.shape[0]
n_candidates = covered.shape[1] # The same as AED.shape[0]
A = pd.DataFrame.to_numpy(covered)

In [14]:
selected_AED_info = []

for K in N:
    h = highspy.Highs()

    n_var = n_candidates + n_cases

    # Create x variables
    x_var_list = []
    for i in range(n_candidates):
        var_name = f'x{i}'
        x_var = h.addVar(lb=0.0, ub=1.0, type = highspy.HighsVarType.kInteger, name=var_name) 
        x_var_list.append(x_var)

    # Create y variables
    y_var_list = []
    for i in range(n_cases):
        var_name = f'y{i}'
        y_var = h.addVar(lb=0.0, ub=1.0, type = highspy.HighsVarType.kInteger, name=var_name)
        y_var_list.append(y_var)


    constr_expr = 0
    for i in range(n_candidates):
        constr_expr += x_var_list[i]
    h.addConstr(constr_expr == K)
    
    constr_expr = 0
    for i in range(n_cases):
        for j in range(n_candidates):
            constr_expr += A[i, j]*x_var_list[j]
        constr_expr -= y_var_list[i]
        h.addConstr(0  <= constr_expr)

    obj_expr = 0
    for i in range(n_cases):
        obj_expr += y_var_list[i]

    h.maximize(obj_expr)
    h.run()
    solution = h.getSolution()

    basis = h.getBasis()
    info = h.getInfo()
    model_status = h.getModelStatus()
    print('Model status = ', h.modelStatusToString(model_status))
    print()
    print('Optimal objective = ', info.objective_function_value)
    print('Iteration count = ', info.simplex_iteration_count)
    print('Primal solution status = ', h.solutionStatusToString(info.primal_solution_status))

    selected = []
    for i in range(n_candidates):
        if h.val(x_var_list[i]) > 0.5:
            selected.append(i)

    for ind in selected:
        row = [K, potential.iloc[ind, 0], potential.iloc[ind, 1], potential.iloc[ind, 2]]
        selected_AED_info.append(row)

selected_AED_pd = pd.DataFrame(selected_AED_info, columns=['N', 'id', 'lattitude', 'longitude'])
selected_AED_pd.to_csv('output_log.csv', index=False)

Model status =  Optimal

Optimal objective =  30.0
Iteration count =  54
Primal solution status =  Feasible
